In [1]:
import pandas as pd
import numpy as np


In [2]:
idx = pd.IndexSlice
pd.options.display.float_format = '{:,.3f}'.format

In [3]:
df = pd.read_pickle('E:/Data/DataSQL/df_merge')
col = pd.read_excel('E:/Data/6.Python/VND-col.xlsx')
sql_col = pd.read_excel('E:/Data/6.Python/SQL-col.xlsx')
nganh = pd.read_excel('E:/Data/NganhICB.xlsx')
list_ticker = list(nganh['ticker'].values)

In [40]:
def add_ratios(x):
    x['jv_income'] = x['jv_inc1'] + x['jv_inc2']
    x['cip'] = x['cip1']
    x['tax_rate'] = 1 - x['net_income'] / (1+x['pretax_inc'])
    x['op'] = x['gp'] - np.abs(x['admin_exp']) - np.abs(x['sel_exp'])
    x['op1'] = x['op'] + x['jv_income']
    x['EBT'] = x['op'] - np.abs(x['int_exp'])
    x['fin_income'] = x['fin_rev'] - (np.abs(x['fin_exp']) - np.abs(x['int_exp']))
    x['other_income'] = x['other_inc']
    # x['bs_gross_fa'] = x['gross_tfa'] + x['gross_ifa'] + x['gross_lease'] + x['gross_ip']
    x['bs_fa'] = x['net_tfa'] + x['net_ifa'] + x['net_ip'] + x['net_lease']
    x['bs_cash'] = (x['cce'] + x['net_st_inv'])
    x['bs_capex'] = -(x.sort_index(level=[0, 1]).groupby(level=0)["bs_fa"].diff(periods=1) +
                      x.sort_index(level=[0, 1]).groupby(level=0)["cip"].diff(periods=1))
    x['bs_ar'] = x['st_ar']
    x['bs_ap'] = x['st_trade_ap']
    x['bs_cust_pre'] = x['st_prepaid_cust'] + x['lt_prepaid_cust'] + x['st_unrev'] + x['lt_unrev']
    x['debt'] = x['st_debt'] + x['lt_debt']
    x['core_e'] = x['op'] * (1 - x['tax_rate'])
    x['op2'] = x['core_e'] + x['jv_income']
    x['netcash'] = (x['bs_cash'] - x['cl'])
    x['ic'] = x['te'] + x['debt']
    x['other_asset'] = x['ta'] - x['bs_cash'] - x['bs_ar'] - x['net_inven'] - x['bs_fa'] - x['cip']-x['lt_inv']
    x['other_lia'] = x['tl'] - x['debt'] - x['bs_ap'] - x['bs_cust_pre']
    x['other_equity'] = x['te'] - x['re'] - x['cap_e'] - x['bs_treasury']
    x['div'] = x['cf_div'] + x['cf_treasury']
    x['cf_delta_debt'] = x['cf_divay'] + x['cf_debt_paid'] + x['cf_lease_paid']
    x['cf_khac'] = (x['cfo'] + x['cfi'] + x['cff']) - (x['cf_capex'] + x['net_income'] + x['cf_dep']
                                                       + x['cf_e_raise'] + x['cf_delta_debt'] + x['div'])
add_ratios(df)

In [41]:
list_chitieu = ['rev','gp','sel_exp','admin_exp','net_income','ta','te','tl','bs_fa','cip','bs_cash','st_debt','lt_debt']
df.loc[idx['HPG',:,5],list_chitieu].to_excel('HPG.xlsx')

In [55]:
list_thep = nganh[(nganh['ICB4']=='Thép và sản phẩm thép L4')&(nganh['ticker']!='HTS')&(nganh['ticker']!='ATP')]['ticker']

In [56]:
x = df.loc[idx[list_thep,:,5],['rev','gp','net_income']].unstack([0,2]).sort_index(level=[0,1])/10**9
x
x.to_excel('thep.xlsx')

In [51]:
df.loc['VIS']

acc_ifa acc_ip acc_lease                acc_tfa   
year quarter                                                              
2005 5.000                0.000  0.000     0.000    -64,541,803,734.000  \
2006 5.000                0.000  0.000     0.000    -92,669,609,289.000   
2007 1.000                0.000  0.000     0.000    -98,664,620,956.000   
     2.000                0.000  0.000     0.000   -104,227,846,628.000   
     3.000                0.000  0.000     0.000   -110,321,325,446.000   
...                         ...    ...       ...                    ...   
2021 4.000     -799,999,971.000  0.000     0.000 -1,275,573,953,251.000   
     5.000     -799,999,971.000  0.000     0.000 -1,275,573,953,251.000   
     6.000   -4,510,865,231.000  0.000     0.000 -1,259,178,352,938.000   
     9.000   -4,648,530,191.000  0.000     0.000 -1,268,941,812,447.000   
2022 1.000     -981,964,968.000  0.000     0.000 -1,280,799,284,052.000   

                       admin_exp bs_jv bs_mi bs_treasury   
year quarter                                               
2005 5.000    -6,339,989,920.000 0.000 0.000       0.000  \
2006 5.000    -8,330,871,821.000 0.000 0.000       0.000   
2007 1.000    -1,914,518,576.000 0.000 0.000       0.000   
     2.000    -3,370,301,188.000 0.000 0.000       0.000   
     3.000    -1,952,152,064.000 0.000 0.000       0.000   
...                          ...   ...   ...         ...   
2021 4.000   -13,239,948,663.000 0.000 0.000       0.000   
     5.000   -54,130,488,918.000 0.000 0.000       0.000   
     6.000   -26,867,935,097.000 0.000 0.000       0.000   
     9.000   -40,890,540,255.000 0.000 0.000       0.000   
2022 1.000   -14,992,657,792.000 0.000 0.000       0.000   

                                ca               cap_e  ...   
year quarter                                            ...   
2005 5.000     486,943,128,261.000  75,978,200,000.000  ...  \
2006 5.000     551,664,284,397.000 100,000,000,000.000  ...   
2007 1.000     501,468,661,408.000 100,000,000,000.000  ...   
     2.000     421,699,682,977.000 100,000,000,000.000  ...   
     3.000     706,996,660,303.000 150,000,000,000.000  ...   
...                            ...                 ...  ...   
2021 4.000   2,721,434,268,968.000 738,303,930,000.000  ...   
     5.000   2,721,434,268,968.000 738,303,930,000.000  ...   
     6.000   2,700,573,794,855.000 738,303,930,000.000  ...   
     9.000   2,754,746,724,992.000 738,303,930,000.000  ...   
2022 1.000   3,157,538,650,694.000 738,303,930,000.000  ...   

                           core_e                op2                netcash   
year quarter                                                                  
2005 5.000     41,262,351,415.674 41,262,351,415.674   -387,757,245,238.000  \
2006 5.000     45,950,881,509.603 45,950,881,509.603   -421,438,215,189.000   
2007 1.000     11,747,713,536.563 11,747,713,536.563   -454,914,983,170.000   
     2.000     19,040,677,699.584 19,040,677,699.584   -359,151,918,504.000   
     3.000     15,719,428,150.804 15,719,428,150.804   -575,468,208,058.000   
...                           ...                ...                    ...   
2021 4.000   -103,942,266,816.546                NaN -2,275,941,950,649.000   
     5.000   -106,498,083,267.110                NaN -2,275,941,950,649.000   
     6.000     81,961,932,184.630 81,961,932,184.630 -1,792,779,982,366.000   
     9.000     -2,742,237,775.225                NaN -2,175,973,117,857.000   
2022 1.000    -19,138,225,180.654                NaN -2,362,840,193,788.000   

                                ic         other_asset           other_lia   
year quarter                                                                 
2005 5.000     673,436,225,934.000  18,505,378,188.000  19,627,870,583.000  \
2006 5.000     712,134,892,175.000  38,045,910,818.000  38,679,034,806.000   
2007 1.000     561,394,601,298.000  40,977,973,543.000 132,864,596,642.000   
     2.000     577,333,216,30